# Getting Started with PrimeHub Python SDK
PrimeHub Python SDK makes you automation with PrimeHub Platform.

In order to make the SDK working, you have to

- install the library with pip
- create a config file in the ~/.primehub/config.json

# Part 1: prerequisite: Configure the environment.

## 1. Install with pip
Let's install PrimeHub Python SDK with pip.

In [1]:
!pip install primehub-python-sdk

## 2. Request the API Token
In order to get the token, you have to have an account in the PrimeHub cluster, the following process will ask you loing with your account.



In [2]:
# PLEASE UPDATE PRIMEHUB_CLUSTER to your cluster
PRIMEHUB_CLUSTER = 'http://primehub-python-sdk.primehub.io'

In [3]:
import os
from primehub import PrimeHub, PrimeHubConfig

ph = PrimeHub(PrimeHubConfig())
if not os.path.isfile(os.path.join(os.getenv("HOME"), ".primehub/config.json")):
    ph.config.generate(PRIMEHUB_CLUSTER)

In [4]:
ph = PrimeHub(PrimeHubConfig())
if ph.is_ready():
    print("PrimeHub Python SDK setup successfully")
    print("Current Group:", ph.primehub_config.current_group)
else:
    print("PrimeHub Python SDK couldn't get the group information, please check the configuration.")

PrimeHub Python SDK setup successfully
Current Group: {'id': '11945641-279a-4042-8688-ef88769c21ca', 'name': 'demo_showcase', 'displayName': 'demo_showcase'}


## 3. Check the account is Admin account

Use `ph.me.me` to know that the account is admin account.

In [5]:
account_information = ph.me.me()
account_information['isAdmin']

True

## 4. Clean up the previous testing data.

We need to clean up the previous data to prevent the notebook error,

In [6]:
import pandas as pd

# Part 2: End-User Journey or End2End ML Journey
We will test:

- Submit a Job/schedule job
- List running jobs(status) submitted by the user
- List registered models with all of versions
- Submit a registered model as a deployment
- Update the deployment with other version of the model

### 1. Submit a Job/schedule job

In [8]:
# Submit a job
config = {
    "instanceType": "cpu-1",
    "image": "base-notebook",
    "displayName": "short-job",
    "command": "echo \"test1\"\necho \"test2\""
}

short_job = ph.jobs.submit(config)
print(short_job)

# Wait the job to be done
print('[ Waiting ]')
ph.jobs.wait(short_job['id'])
print('[ Job Done ]')

# Get logs
logs = ph.jobs.logs(short_job['id'])
print('[ Job Logs ]')
for l in logs:
    print(l.decode("utf-8"))

{'id': 'job-202211280818-5ojjpm', 'displayName': 'short-job', 'cancel': False, 'command': 'echo "test1"\necho "test2"', 'groupId': '11945641-279a-4042-8688-ef88769c21ca', 'groupName': 'demo_showcase', 'image': 'base-notebook', 'instanceType': {'id': 'cpu-1', 'name': 'cpu-1', 'displayName': 'CPU 1', 'cpuLimit': 1, 'memoryLimit': 2, 'gpuLimit': 0}, 'userId': '35985b6b-21c9-4362-902d-4d6bb89074bf', 'userName': 'simonliu@infuseai.io', 'phase': 'Pending', 'reason': None, 'message': None, 'createTime': '2022-11-28T08:18:18Z', 'startTime': None, 'finishTime': None, 'recurrence': None}
[ Waiting ]
[ Job Done ]
[ Job Logs ]
test1
test2
Artifacts: no artifact found



In [14]:
# Create a recurring job that runs at every 4 AM
config = {
    "instanceType": "cpu-1",
    "image": "base-notebook",
    "displayName": "test",
    "command": "echo \"test!\"",
    "recurrence": {
        "type":"daily",
        "cron":""
    }
}

recurring_job = ph.recurring_jobs.create(config)
print(recurring_job)

{'id': 'recurrence-9oybl9', 'displayName': 'test', 'recurrence': {'type': 'daily', 'cron': ''}, 'invalid': False, 'message': None, 'command': 'echo "test!"', 'groupId': '11945641-279a-4042-8688-ef88769c21ca', 'groupName': 'demo_showcase', 'image': 'base-notebook', 'instanceType': {'id': 'cpu-1', 'name': 'cpu-1', 'displayName': 'CPU 1', 'cpuLimit': 1, 'memoryLimit': 2, 'gpuLimit': 0}, 'userId': '35985b6b-21c9-4362-902d-4d6bb89074bf', 'userName': 'simonliu@infuseai.io', 'nextRunTime': None}


In [15]:
df = pd.DataFrame.from_records(list(ph.recurring_jobs.list()))
df

,id,displayName,recurrence,invalid,message,command,groupId,groupName,image,instanceType,userId,userName,nextRunTime
0,recurrence-9oybl9,test,"{'type': 'daily', 'cron': ''}",False,None,"echo ""test!""",11945641-279a-4042-8688-ef88769c21ca,demo_showcase,base-notebook,"{'id': 'cpu-1', 'name': 'cpu-1', 'displayName'...",35985b6b-21c9-4362-902d-4d6bb89074bf,simonliu@infuseai.io,2022-11-28T20:00:00Z


In [16]:
ph.recurring_jobs.delete(recurring_job['id'])

{'id': 'recurrence-9oybl9'}

### 2. List running jobs(status) submitted by the user


In [41]:
df = pd.DataFrame.from_records(list(ph.jobs.list()))
df[(df['phase']=="Running") & (df['userName']=="simonliu@infuseai.io")]

,id,displayName,cancel,command,groupId,groupName,image,instanceType,userId,userName,phase,reason,message,createTime,startTime,finishTime,recurrence
0,job-202211280835-c4lbkd,short-job,None,"echo ""test1""\necho ""test2""",11945641-279a-4042-8688-ef88769c21ca,demo_showcase,base-notebook,"{'id': 'cpu-1', 'name': 'cpu-1', 'displayName'...",35985b6b-21c9-4362-902d-4d6bb89074bf,simonliu@infuseai.io,Running,PodRunning,Job is currently running,2022-11-28T08:35:27Z,2022-11-28T08:35:32Z,None,None


### 3. List registered models with all of versions


In [73]:
df = pd.DataFrame.from_records(ph.models.list())
df

,name,creationTimestamp,lastUpdatedTimestamp,description,latestVersion
0,mnist,2021-09-30 04:48:39,2021-09-30 05:15:16,None,2
1,tf-screw-model,2021-11-18 16:43:53,2022-01-17 04:01:03,None,1
2,example,2022-07-14 06:28:53,2022-07-14 06:28:54,None,1
3,mask-detection-model,2022-10-05 05:20:37,2022-11-14 06:50:29,None,4


In [72]:
df = pd.DataFrame.from_records(ph.models.list_versions('mnist'))
df

,name,version,creationTimestamp,lastUpdatedTimestamp,deployedBy
0,mnist,1,2021-09-30 04:48:39,2021-09-30 04:48:39,"[{'id': 'mnist-1209-a9wjo', 'name': 'mnist-120..."
1,mnist,2,2021-09-30 05:15:16,2021-09-30 05:15:16,"[{'id': 'mnist-9ofpg', 'name': 'mnist'}]"


### 4. Submit a registered model as a deployment


In [75]:
# Create a deployment
config = {
    "id": "deploymeny-from-sdk-zxy",
    "name": "deploymeny-from-sdk",
    "modelImage": "base-notebook",
    "modelURI": "test/module/uri",
    "instanceType": "cpu-1",
    "replicas": 1
}

deployment = ph.deployments.create(config)
print(deployment)

{'id': 'deploymeny-from-sdk-zxy', 'name': 'deploymeny-from-sdk', 'modelImage': 'base-notebook', 'imagePullSecret': None, 'description': None, 'replicas': 1, 'stop': False, 'endpointAccessType': 'public', 'endpointClients': [], 'status': 'Deploying', 'endpoint': None, 'availableReplicas': None, 'message': None, 'pods': []}


In [76]:
df = pd.DataFrame.from_records(ph.deployments.list())
df[df['name'] == "deploymeny-from-sdk"]

,id,name,modelImage,imagePullSecret,description,replicas,stop,endpointAccessType,endpointClients,status,endpoint,availableReplicas,message,pods
0,deploymeny-from-sdk-zxy,deploymeny-from-sdk,base-notebook,None,None,1,False,public,[],Deploying,https://c.demo.primehub.io/deployment/deployme...,NaN,Failed because of wrong image settings.\npod[d...,[{'name': 'deploy-deploymeny-from-sdk-zxy-cf9b...


### 5. Update the deployment with other version of the model

In [77]:
# Update the deployment
update_config = {
    "replicas": 2
}

ph.deployments.update(deployment['id'], update_config)

{'id': 'deploymeny-from-sdk-zxy',
 'name': 'deploymeny-from-sdk',
 'modelImage': 'base-notebook',
 'imagePullSecret': None,
 'description': None,
 'replicas': 2,
 'stop': False,
 'endpointAccessType': 'public',
 'endpointClients': [],
 'status': 'Deploying',
 'endpoint': 'https://c.demo.primehub.io/deployment/deploymeny-from-sdk-zxy/api/v1.0/predictions',
 'availableReplicas': None,
 'message': 'Failed because of wrong image settings.\npod[deploy-deploymeny-from-sdk-zxy-cf9b9bc6f-fmd7d] failed\n  reason: ContainersNotInitialized, message: containers with incomplete status: [model-storage-initializer]\n  container state: Waiting, reason: ImagePullBackOff, message Back-off pulling image "gcr.io/kfserving/storage-initializer:v0.4.0"',
 'pods': [{'name': 'deploy-deploymeny-from-sdk-zxy-cf9b9bc6f-fmd7d'}]}

In [78]:
df = pd.DataFrame.from_records(ph.deployments.list())
df[df['name'] == "deploymeny-from-sdk"]

,id,name,modelImage,imagePullSecret,description,replicas,stop,endpointAccessType,endpointClients,status,endpoint,availableReplicas,message,pods
0,deploymeny-from-sdk-zxy,deploymeny-from-sdk,base-notebook,None,None,2,False,public,[],Deploying,https://c.demo.primehub.io/deployment/deployme...,NaN,Failed because of wrong image settings.\npod[d...,[{'name': 'deploy-deploymeny-from-sdk-zxy-cf9b...


In [79]:
# Delete a deployment
ph.deployments.delete(deployment['id'])

{'id': 'deploymeny-from-sdk-zxy'}

In [80]:
df = pd.DataFrame.from_records(ph.deployments.list())
df[df['name'] == "deploymeny-from-sdk"]

,id,name,modelImage,imagePullSecret,description,replicas,stop,endpointAccessType,endpointClients,status,endpoint,availableReplicas,message,pods
